# ================================================
# Laboratorio 2 — Similitud de Coseno
# ================================================
Diego Leiva       -   21752   
# Instrucciones
1. **Carga de embeddings**  
   - Utilizar las representaciones vectoriales preentrenadas (*GloVe 100d*).  
   - Verificar que los términos requeridos estén contenidos en el vocabulario.  

2. **Cálculo de vectores de expresión**  
   - Construir expresiones vectoriales simples mediante operaciones aritméticas (suma, resta).  

3. **Medición de similitud**  
   - Calcular la similitud de coseno entre los vectores de la expresión y los vectores de referencia.  
   - Reportar:  
     - Valor de la similitud de coseno.  
     - Normas de cada vector (\(\|\mathbf{u}\|\)).  

4. **Casos de estudio**  
   Realizar caso por caso

5. **Documentación de resultados**  
   - Registrar en el notebook los valores obtenidos de similitud y normas vectoriales para cada caso.  
   - Incluir comentarios breves interpretando los resultados.  

6. **Conclusiones**  
   - Redactar conclusiones **caso por caso** sobre la utilidad de la similitud de coseno.

In [ ]:
import numpy as np
from typing import List, Iterable, Tuple
import gensim.downloader as api

In [ ]:
# ========= 1) Cargar GloVe 100D =========
model = api.load("glove-wiki-gigaword-100")

In [ ]:
# ========= 2) Funciones utilitarias =========
def embedding(word: str) -> np.ndarray:
    """
    Vector de la palabra (lanza KeyError si no está en vocab).
    Args:
        word (str): La palabra a vectorizar.
    Returns:
        np.ndarray: El vector de la palabra.
    """
    return model[word]

def build_vector(plus: List[str], minus: List[str]) -> np.ndarray:
    """
    Suma y resta embeddings explícitamente.
    Args:
        plus (List[str]): Palabras a sumar.
        minus (List[str]): Palabras a restar.
    Returns:
        np.ndarray: El vector resultante.
    """
    dim = model[next(iter(model.key_to_index))].shape[0]
    v = np.zeros(dim, dtype=np.float32)
    for w in plus:
        if w in model:
            v += model[w]
        else:
            print(f"[OOV] '{w}' no está en el vocabulario.")
    for w in minus:
        if w in model:
            v -= model[w]
        else:
            print(f"[OOV] '{w}' no está en el vocabulario.")
    return v

def cos(a: np.ndarray, b: np.ndarray) -> float:
    """
    Calcula la similitud coseno entre dos vectores.
    Args:
        a (np.ndarray): Primer vector.
        b (np.ndarray): Segundo vector.
    Returns:
        float: La similitud coseno entre los vectores.
    """
    na, nb = np.linalg.norm(a), np.linalg.norm(b)
    if na == 0 or nb == 0:
        return float("nan")
    return float(np.dot(a, b) / (na * nb))

def vec_norm(v: np.ndarray) -> float:
    """
    Calcula la norma de un vector.
    Args:
        v (np.ndarray): El vector a evaluar.
    Returns:
        float: La norma del vector.
    """
    return float(np.linalg.norm(v))

def stringify_expr(plus: List[str], minus: List[str]) -> str:
    """
    Convierte la expresión en un string legible.
    Args:
        plus (List[str]): Palabras a sumar.
        minus (List[str]): Palabras a restar.
    Returns:
        str: La expresión en formato string.
    """
    p = " + ".join(plus) if plus else ""
    m = " - " + " - ".join(minus) if minus else ""
    return (p + m).strip() if (p or m) else "(vector nulo)"

def check_vocab(words: Iterable[str]) -> Tuple[set, set]:
    """
    Regresa (presentes, ausentes) en el vocab.
    Args:
        words (Iterable[str]): Palabras a verificar.
    Returns:
        Tuple[set, set]: Conjuntos de palabras presentes y ausentes en el vocab.
    """
    presentes, ausentes = set(), set()
    for w in words:
        (presentes if w in model else ausentes).add(w)
    return presentes, ausentes

def show_vocab_check(title: str, words: Iterable[str]) -> None:
    """
    Muestra las palabras presentes y ausentes en el vocab.
    Args:
        title (str): Título de la verificación.
        words (Iterable[str]): Palabras a verificar.
    """
    presentes, ausentes = check_vocab(words)
    print(f"--- {title} ---")
    print("En vocab:", sorted(presentes))
    if ausentes:
        print("Fuera de vocab (OOV):", sorted(ausentes))
    else:
        print("No hay OOV ✅")
    print()


def report_case(label, plus, minus, target_plus) -> None:
    """
    Reporta un caso de prueba.
    Args:
        label (str): Etiqueta del caso.
        plus (List[str]): Palabras a sumar.
        minus (List[str]): Palabras a restar.
        target_plus (List[str]): Palabras objetivo a sumar.
    """
    expr_vec   = build_vector(plus, minus)
    target_vec = build_vector(target_plus, [])
    print("====", label, "====")
    print("Expresión =", " + ".join(plus) + " - " + " - ".join(minus) if minus else " + ".join(plus))
    print("Target    =", " + ".join(target_plus))
    print("cos(expr, target) =", round(cos(expr_vec, target_vec), 6))
    print("||expr||           =", round(vec_norm(expr_vec), 6))
    print("||target||         =", round(vec_norm(target_vec), 6))
    print()

In [ ]:
# ========= 3) Vecinos más cercanos de una expresión =========
def nearest_neighbors(plus: List[str], minus: List[str], topn: int = 3) -> List[tuple]:
    """
    Encuentra los vecinos más cercanos a una expresión usando similar_by_vector.
    Args:
        plus (List[str]): Palabras a sumar.
        minus (List[str]): Palabras a restar.
        topn (int): Número de vecinos a retornar.
    Returns:
        List[tuple]: Lista de (palabra, similitud, norma_vector).
    """
    expr_vec = build_vector(plus, minus)
    # similar_by_vector calcula coseno internamente contra todo el vocab
    sims = model.similar_by_vector(expr_vec, topn=topn)
    out = []
    for word, sim in sims:
        v = model[word]
        out.append((word, float(sim), vec_norm(v)))
    return out

def report_neighbors(title: str, plus: List[str], minus: List[str], topn: int = 3) -> None:
    """
    Reporta los vecinos más cercanos de una expresión.
    Args:
        title (str): Título de la sección.
        plus (List[str]): Palabras a sumar.
        minus (List[str]): Palabras a restar.
        topn (int): Número de vecinos a mostrar.
    """
    print(f"==== {title} ====")
    print("Expresión =", stringify_expr(plus, minus))
    nn = nearest_neighbors(plus, minus, topn=topn)
    for i, (w, s, n) in enumerate(nn, 1):
        print(f"{i}. {w:15s}  cos = {s:.6f}   ||w|| = {n:.6f}")
    print()

## Caso 1 — Capitales

Construya un vector que represente la relación:  
**paris + italy - france ≈ rome**

1. Defina el vector de la expresión (`expr_vec`).  
2. Defina el vector objetivo (`target_vec`).  
3. Calcule `cos(expr, target)`, `||expr||` y `||target||`.  
4. Escriba su conclusión.  

## Caso 2 — Dictador/País

Construya un vector que represente la relación:  
**hitler + italy - germany**

1. Construya el vector de la expresión.  
2. Encuentre los **3 vecinos más cercanos** (`nearest_neighbors`).  
3. Reporte la similitud coseno con cada vecino.  
4. Escriba su interpretación de los resultados.  

## Caso 3 — Religión/Líder

Construya un vector que represente la relación:  
**christianity ≈ jesus**

1. Construya el vector de la palabra `christianity`.  
2. Encuentre los **3 vecinos más cercanos** (`nearest_neighbors`).  
3. Reporte la similitud coseno con cada vecino.  
4. Escriba su interpretación: ¿aparece `jesus` entre los vecinos?  

## Caso 4 — Opuestos

Analice la relación:  
**good - bad**

1. Construya el vector de la expresión `good - bad`.  
2. Encuentre los **3 vecinos más cercanos** (`nearest_neighbors`).  
3. Reporte la similitud coseno con cada vecino.  
4. Escriba su conclusión sobre si los vecinos reflejan un contraste u oposición semántica.

## Caso 5 — Propio

Defina usted mismo un caso interesante. Ejemplos:  
- `king - man + woman`  
- `tokyo + france - japan`  
- `apple - technology`  

1. Construya la expresión vectorial que haya definido.  
2. Encuentre los **3 vecinos más cercanos** (`nearest_neighbors`).  
3. Reporte la similitud coseno con cada vecino.  
4. Redacte una conclusión explicando si los resultados corresponden a su hipótesis inicial.